In [30]:
import sys
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install nba_api
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install sqlalchemy




  Obtaining dependency information for sqlalchemy from https://files.pythonhosted.org/packages/68/89/247a6a5bfcc5f2f27b08ce7a1fbc2ee9e54fa09ccc9fa48e8c0e08e78cbd/SQLAlchemy-2.0.21-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.2.0 from https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)
     ---------------------------------------- 0.0/192.5 kB ? eta -:--:--
     -- ------------------------------------- 10.2/192.5 kB ? eta -:--:--
     ------------- ----------------------- 71.7/192.5 kB 975.2 kB/s eta 0:00:01
     -------------------------------------- 192.5/192.5 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------- ------------------------ 0.8/2.0 MB 25.1 MB/s eta 0:00:01
   ---------------------- ------

In [291]:
from dotenv import load_dotenv
import os
load_dotenv()
import mysql.connector as sql
import pandas as pd
from sqlalchemy import create_engine

host=os.getenv("DB_HOST")
database=os.getenv("DB_NAME")
user=os.getenv("DB_USERNAME")
password=os.getenv("DB_PASSWORD")


conn = sql.connect(host=os.getenv("DB_HOST"),
                   database=os.getenv("DB_NAME"),
                   user=os.getenv("DB_USERNAME"),
                   password=os.getenv("DB_PASSWORD"),
                    )
mycursor = conn.cursor()
# Create the SQLAlchemy engine using the MySQL connection
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [270]:
query = """ select * from vw_wl_features """
pd.read_sql_query(query,engine)

,team_id,game_id,game_date,last_game_oe,last_game_home_win_pctg,num_rest_days,last_game_away_win_pctg,last_game_rolling_scoring_margin,last_game_rolling_oe
0,1610612737,22201216,2023-04-09,0.588652,0.585366,2.0,0.425000,10.333333,0.602885
1,1610612766,22201218,2023-04-09,0.500000,0.317073,2.0,0.325000,-14.333333,0.538177
2,1610612765,22201223,2023-04-09,0.581197,0.219512,2.0,0.200000,-7.000000,0.575409
3,1610612759,22201224,2023-04-09,0.636364,0.341463,1.0,0.175000,-13.000000,0.573532
4,1610612758,22201227,2023-04-09,0.534483,0.560976,2.0,0.625000,-2.666667,0.597749
5,1610612762,22201228,2023-04-09,0.573643,0.560976,1.0,0.350000,-4.666667,0.572762
6,1610612753,22201219,2023-04-09,0.436364,0.487805,2.0,0.350000,-15.000000,0.503979
7,1610612740,22201225,2023-04-09,0.605263,0.658537,2.0,0.375000,-1.000000,0.577084
8,1610612755,22201217,2023-04-09,0.591549,0.707317,2.0,0.600000,-7.000000,0.570550
9,1610612754,22201220,2023-04-09,0.573770,0.487805,2.0,0.350000,-8.666667,0.568709


In [286]:
query = """ select cast(gameId as signed) as GAME_ID
                  ,w_home as home_win_flag
             from 
                stg_nba_schedule sns
             join
                stg_game_stats sgs on sns.gameId = sgs.game_id """
pd.read_sql_query(query,engine)



,CITY,NICKNAME,TEAM_ID,W,L,W_HOME,L_HOME,W_ROAD,L_ROAD,TEAM_TURNOVERS,...,BLK,PTS,AVG_REB,AVG_PTS,DQ,OFFENSIVE_EFFICIENCY,SCORING_MARGIN,SEASON,GAME_DATE,GAME_ID
0,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,8,124,63.0,124.0,0,0.653226,29.0,2020-21,2021-05-16,22001066
1,OPPONENTS,Rockets,1610612745,0,1,0,0,0,1,1,...,5,95,51.0,95.0,0,0.459016,-29.0,2020-21,2021-05-16,22001066
2,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,4,116,58.0,116.0,0,0.555556,23.0,2020-21,2021-05-13,22001049
3,OPPONENTS,Magic,1610612753,0,1,0,0,0,1,0,...,6,93,62.0,93.0,0,0.517241,-23.0,2020-21,2021-05-13,22001049
4,Atlanta,Hawks,1610612737,1,0,1,0,0,0,0,...,5,120,56.0,120.0,0,0.573913,4.0,2020-21,2021-05-12,22001042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7075,OPPONENTS,Pistons,1610612765,0,1,0,0,0,1,0,...,3,106,50.0,106.0,0,0.550459,-11.0,2022-23,2023-02-27,22200919
7076,Detroit,Pistons,1610612765,1,0,1,0,0,0,1,...,3,118,60.0,118.0,0,0.572581,6.0,2022-23,2023-02-03,22200785
7077,OPPONENTS,Hornets,1610612766,0,1,0,0,0,1,1,...,2,112,46.0,112.0,1,0.525424,-6.0,2022-23,2023-02-03,22200785
7078,Charlotte,Hornets,1610612766,0,1,0,1,0,0,0,...,6,134,65.0,134.0,1,0.552846,-7.0,2022-23,2022-12-14,22200414


In [2]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import cumestatsteamgames, cumestatsteam, gamerotation
import pandas as pd
import numpy as np
import json
import difflib
import time
import requests

# Retry Wrapper 
def retry(func, retries=3):
    def retry_wrapper(*args, **kwargs):
        attempts = 0
        while attempts < retries:
            try:
                return func(*args, **kwargs)
            except requests.exceptions.RequestException as e:
                print(e)
                time.sleep(30)
                attempts += 1

    return retry_wrapper

# Get Single Game aggregation columns

def getSingleGameMetrics(gameID,homeTeamID,awayTeamID,awayTeamNickname,seasonYear,gameDate):

    @retry
    def getGameStats(teamID,gameID,seasonYear):
        #print(f'game_id: {gameID} team_id: {teamID} season:{seasonYear}')
        gameStats = cumestatsteam.CumeStatsTeam(game_ids='00'+str(gameID),league_id ="00",
                                               season=seasonYear,season_type_all_star="Regular Season",
                                               team_id = teamID).get_normalized_json()

        #print(gameStats)
        gameStats = pd.DataFrame(json.loads(gameStats)['TotalTeamStats'])

        return gameStats

    data = getGameStats(homeTeamID,gameID,seasonYear)
    #print(data)
    data.at[1,'NICKNAME'] = awayTeamNickname
    data.at[1,'TEAM_ID'] = awayTeamID
    data.at[1,'OFFENSIVE_EFFICIENCY'] = (data.at[1,'FG'] + data.at[1,'AST'])/(data.at[1,'FGA'] - data.at[1,'OFF_REB'] + data.at[1,'AST'] + data.at[1,'TOTAL_TURNOVERS'])
    data.at[1,'SCORING_MARGIN'] = data.at[1,'PTS'] - data.at[0,'PTS']

    data.at[0,'OFFENSIVE_EFFICIENCY'] = (data.at[0,'FG'] + data.at[0,'AST'])/(data.at[0,'FGA'] - data.at[0,'OFF_REB'] + data.at[0,'AST'] + data.at[0,'TOTAL_TURNOVERS'])
    data.at[0,'SCORING_MARGIN'] = data.at[0,'PTS'] - data.at[1,'PTS']

    data['SEASON'] = seasonYear
    data['GAME_DATE'] = gameDate
    data['GAME_ID'] = gameID

    return data

In [7]:
sql_query = """ SELECT 
                    sss.* 
                FROM 
                    stg_season_schedule sss
                LEFT JOIN 
                    stg_game_stats sgs1 on sss.game_id = sgs1.game_id and sss.home_team_id = sgs1.team_id
                LEFT JOIN 
                    stg_game_stats sgs2 on sss.game_id = sgs2.game_id and sss.away_team_id = sgs2.team_id
                WHERE 
                    sgs1.team_id is null
                      """

# Execute the SQL query and return the result as a DataFrame
df = pd.read_sql_query(sql_query, engine)
df

,MATCHUP,GAME_ID,SEASON,GAME_DATE,HOME_TEAM_NICKNAME,HOME_TEAM_ID,AWAY_TEAM_NICKNAME,AWAY_TEAM_ID


In [4]:
failed_attempts = 0
for _,row in df.iterrows():
    try:
        getSingleGameMetrics(row['GAME_ID'],row['HOME_TEAM_ID'],row['AWAY_TEAM_ID'],row['AWAY_TEAM_NICKNAME'],row['SEASON'],row['GAME_DATE']).to_sql('stg_game_stats', engine, if_exists='append', index=False, method='multi')
    except:
        print(f'failed attempt number {failed_attempts}')
        failed_attempts+= 1
        continue

In [272]:
url = "https://nba-prediction-api.onrender.com/all"
response = requests.get(url)
#historic_games = pd.read_json(response.json(),orient='records')


In [278]:
response.content

b'Internal Server Error'

In [280]:
query = """ select * from fct_wl_features"""
df = pd.read_sql_query(query,engine)
df

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1146 (42S02): target: basketballdb.-.primary: vttablet: Table 'basketballdb.fct_wl_features' doesn't exist (errno 1146) (sqlstate 42S02) (CallerID: 1s12yv1z7l4mbixy53db): Sql: "select * from fct_wl_features", BindVars: {REDACTED}
[SQL:  select * from fct_wl_features]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [207]:
import pandas as pd 
import requests

url = "https://nba-prediction-api.onrender.com/all"
response = requests.get(url)
df = pd.read_json(response.json(),orient='records')
df

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,HOME_TEAM_ID,GAME_ID,SEASON,GAME_DATE,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE,AWAY_TEAM_ID
0,0.535088,NaN,2.0,1.000000,1.000000,15.000000,0.549056,1,1610612737,22000041,2020-21,2020-12-28,0.470588,0.000000,2.0,0.000000,0.000000,-9.500000,0.519910,1610612765
1,0.567797,1.000000,1.0,0.750000,0.800000,7.333333,0.592552,0,1610612737,22000083,2020-21,2021-01-02,0.531532,0.666667,2.0,0.500000,0.600000,-1.666667,0.549664,1610612739
2,0.500000,0.500000,2.0,0.750000,0.666667,3.000000,0.559789,0,1610612737,22000095,2020-21,2021-01-04,0.580357,0.500000,2.0,0.500000,0.500000,-1.333333,0.510969,1610612752
3,0.561404,0.333333,2.0,0.750000,0.571429,2.666667,0.543067,0,1610612737,22000110,2020-21,2021-01-06,0.528000,0.333333,2.0,0.250000,0.285714,-15.666667,0.533390,1610612766
4,0.531532,0.250000,2.0,0.600000,0.444444,-7.000000,0.530978,1,1610612737,22000153,2020-21,2021-01-11,0.536585,0.833333,2.0,0.500000,0.700000,-6.666667,0.559370,1610612755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,0.609756,0.333333,2.0,0.307692,0.320000,-1.666667,0.566357,1,1610612766,22201113,2022-23,2023-03-26,0.584071,0.578947,2.0,0.388889,0.486486,-4.666667,0.580104,1610612742
3536,0.656489,0.351351,3.0,0.325000,0.337662,5.666667,0.613386,0,1610612766,22201148,2022-23,2023-03-31,0.598425,0.526316,2.0,0.421053,0.473684,-4.333333,0.621818,1610612741
3537,0.508197,0.342105,2.0,0.325000,0.333333,-7.000000,0.579533,0,1610612766,22201163,2022-23,2023-04-02,0.623853,0.650000,2.0,0.324324,0.493506,5.666667,0.617840,1610612761
3538,0.571429,0.333333,2.0,0.325000,0.329114,-15.666667,0.578705,0,1610612766,22201176,2022-23,2023-04-04,0.696296,0.650000,2.0,0.342105,0.500000,9.000000,0.655736,1610612761


In [300]:
query = "select * from wrk_wl_features where season = '2022-23'"
pd.read_sql_query(query,engine).dropna()

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,HOME_TEAM_ID,GAME_ID,SEASON,GAME_DATE,AWAY_LAST_GAME_OE,AWAY_LAST_GAME_HOME_WIN_PCTG,AWAY_NUM_REST_DAYS,AWAY_LAST_GAME_AWAY_WIN_PCTG,AWAY_LAST_GAME_TOTAL_WIN_PCTG,AWAY_LAST_GAME_ROLLING_SCORING_MARGIN,AWAY_LAST_GAME_ROLLING_OE,AWAY_TEAM_ID
3,0.566929,0.666667,3.0,0.600000,0.625000,-8.333333,0.557683,1,1610612737,22200134,2022-23,2022-11-05,0.580357,0.666667,1.0,0.600000,0.625000,9.000000,0.595040,1610612740
4,0.568182,0.750000,2.0,0.600000,0.666667,-4.666667,0.559942,1,1610612737,22200149,2022-23,2022-11-07,0.535088,1.000000,2.0,1.000000,1.000000,17.333333,0.563690,1610612749
5,0.565574,0.800000,2.0,0.600000,0.700000,11.666667,0.566895,0,1610612737,22200162,2022-23,2022-11-09,0.638462,1.000000,2.0,0.571429,0.750000,15.000000,0.595777,1610612762
6,0.590551,0.666667,1.0,0.600000,0.636364,5.333333,0.574769,1,1610612737,22200173,2022-23,2022-11-10,0.514563,0.333333,3.0,0.600000,0.454545,0.000000,0.531692,1610612755
7,0.603604,0.714286,2.0,0.571429,0.642857,4.000000,0.543206,0,1610612737,22200214,2022-23,2022-11-16,0.568000,0.857143,2.0,0.714286,0.785714,10.666667,0.606909,1610612738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,0.609756,0.333333,2.0,0.307692,0.320000,-1.666667,0.566357,1,1610612766,22201113,2022-23,2023-03-26,0.584071,0.578947,2.0,0.388889,0.486486,-4.666667,0.580104,1610612742
1226,0.656489,0.351351,3.0,0.325000,0.337662,5.666667,0.613386,0,1610612766,22201148,2022-23,2023-03-31,0.598425,0.526316,2.0,0.421053,0.473684,-4.333333,0.621818,1610612741
1227,0.508197,0.342105,2.0,0.325000,0.333333,-7.000000,0.579533,0,1610612766,22201163,2022-23,2023-04-02,0.623853,0.650000,2.0,0.324324,0.493506,5.666667,0.617840,1610612761
1228,0.571429,0.333333,2.0,0.325000,0.329114,-15.666667,0.578705,0,1610612766,22201176,2022-23,2023-04-04,0.696296,0.650000,2.0,0.342105,0.500000,9.000000,0.655736,1610612761


In [334]:
import requests 
import pandas as pd

url = "https://nba-prediction-api.onrender.com/test_data"
response = requests.get(url)
test_data = pd.read_json(response.json(),orient='records').dropna().sort_values('GAME_DATE',ascending=True)

In [336]:
from sklearn import preprocessing


X_test = test_data[['HOME_LAST_GAME_OE', 'HOME_LAST_GAME_HOME_WIN_PCTG',
       'HOME_NUM_REST_DAYS', 'HOME_LAST_GAME_AWAY_WIN_PCTG',
       'HOME_LAST_GAME_TOTAL_WIN_PCTG',
       'HOME_LAST_GAME_ROLLING_SCORING_MARGIN', 'HOME_LAST_GAME_ROLLING_OE',
       'AWAY_LAST_GAME_OE', 'AWAY_LAST_GAME_HOME_WIN_PCTG',
       'AWAY_NUM_REST_DAYS', 'AWAY_LAST_GAME_AWAY_WIN_PCTG',
       'AWAY_LAST_GAME_TOTAL_WIN_PCTG',
       'AWAY_LAST_GAME_ROLLING_SCORING_MARGIN', 'AWAY_LAST_GAME_ROLLING_OE']]
scaler = preprocessing.StandardScaler()
scaler.fit(X_test)
scaled_data = scaler.transform(X_test) 

test_data['PREDICTION'] = model.predict(scaled_data)

test_data['TRUE_POSITIVE_BOOL'] = ((test_data['PREDICTION'] == 1) & (test_data['HOME_W'] == 1))
test_data['TRUE_NEGATIVE_BOOL'] = ((test_data['PREDICTION'] == 0) & (test_data['HOME_W'] == 0))
test_data['FALSE_POSITIVE_BOOL'] = ((test_data['PREDICTION'] == 1) & (test_data['HOME_W'] == 0))
test_data['FALSE_NEGATIVE_BOOL'] = ((test_data['PREDICTION'] == 0) & (test_data['HOME_W'] == 1))
test_data['TRUE_POSITIVE_TOTAL_CNT'] = test_data['TRUE_POSITIVE_BOOL'].cumsum()
test_data['TRUE_NEGATIVE_TOTAL_CNT'] = test_data['TRUE_NEGATIVE_BOOL'].cumsum()
test_data['FALSE_POSITIVE_TOTAL_CNT'] = test_data['FALSE_POSITIVE_BOOL'].cumsum()
test_data['FALSE_NEGATIVE_TOTAL_CNT'] = test_data['FALSE_NEGATIVE_BOOL'].cumsum()

#PRECISION/ACCURACY
test_data['ACCURACY'] = (test_data['TRUE_NEGATIVE_TOTAL_CNT'] + test_data['TRUE_POSITIVE_TOTAL_CNT'])/(test_data['TRUE_NEGATIVE_TOTAL_CNT'] + test_data['TRUE_POSITIVE_TOTAL_CNT'] + test_data['FALSE_NEGATIVE_TOTAL_CNT'] + test_data['FALSE_POSITIVE_TOTAL_CNT']) 
test_data['PRECISION'] = test_data['TRUE_POSITIVE_TOTAL_CNT'] / (test_data['TRUE_POSITIVE_TOTAL_CNT'] + test_data['FALSE_POSITIVE_TOTAL_CNT']) 
test_data

,HOME_LAST_GAME_OE,HOME_LAST_GAME_HOME_WIN_PCTG,HOME_NUM_REST_DAYS,HOME_LAST_GAME_AWAY_WIN_PCTG,HOME_LAST_GAME_TOTAL_WIN_PCTG,HOME_LAST_GAME_ROLLING_SCORING_MARGIN,HOME_LAST_GAME_ROLLING_OE,HOME_W,HOME_TEAM_ID,GAME_ID,...,TRUE_POSITIVE_BOOL,TRUE_NEGATIVE_BOOL,FALSE_POSITIVE_BOOL,FALSE_NEGATIVE_BOOL,TRUE_POSITIVE_TOTAL_CNT,TRUE_NEGATIVE_TOTAL_CNT,FALSE_POSITIVE_TOTAL_CNT,FALSE_NEGATIVE_TOTAL_CNT,ACCURACY,PRECISION
739,0.509615,0.000,2.0,0.000000,0.000000,-5.500000,0.518959,0,1610612755,22200028,...,False,False,True,False,0,0,1,0,0.000000,0.000000
411,0.470588,0.000,3.0,0.000000,0.000000,-10.000000,0.477602,0,1610612747,22200037,...,False,True,False,False,0,1,1,0,0.500000,0.000000
616,0.632000,1.000,3.0,0.000000,0.500000,10.500000,0.590809,1,1610612752,22200046,...,True,False,False,False,1,1,1,0,0.666667,0.500000
165,0.514563,0.000,2.0,0.500000,0.333333,-8.666667,0.522410,1,1610612741,22200047,...,False,False,False,True,1,1,1,1,0.500000,0.500000
329,0.500000,0.000,2.0,0.000000,0.000000,-12.333333,0.531298,1,1610612745,22200048,...,False,False,False,True,1,1,1,2,0.400000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.603448,0.525,2.0,0.439024,0.481481,-9.333333,0.555028,1,1610612741,22201223,...,True,False,False,False,477,235,257,210,0.603902,0.649864
983,0.557377,0.575,3.0,0.390244,0.481481,-1.666667,0.566125,1,1610612760,22201226,...,False,False,False,True,477,235,257,211,0.603390,0.649864
1024,0.554545,0.650,2.0,0.341463,0.493827,-1.000000,0.564518,1,1610612761,22201221,...,False,False,False,True,477,235,257,212,0.602879,0.649864
286,0.557252,0.825,1.0,0.463415,0.641975,-9.666667,0.576066,1,1610612743,22201227,...,False,False,False,True,477,235,257,213,0.602369,0.649864


In [1]:
import pandas as pd

students = {"name" : ["Tim Voss", "Nicole Johnson", "Elsa Williams", "John James", "Catherine Jones"], "age" : [19, 20, 21, 20, 23], "favorite_color" : ["red", "yellow", "green", "blue", "green"], "grade" : [91, 95, 82, 75, 93]}

students_df = pd.DataFrame(students)


In [17]:
students_df[((students_df['favorite_color'] == 'red') | (students_df['favorite_color'] == 'green')) & (students_df['grade'] > 90) ]

,name,age,favorite_color,grade
0,Tim Voss,19,red,91
4,Catherine Jones,23,green,93
